In [ ]:
import MDAnalysis as md
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt

u = md.Universe('simulacao/dodab/slipids2/mix/md.gro', 'simulacao/dodab/slipids2/mix/skip100.xtc')
frames = len(u.trajectory)
n_atoms = u.select_atoms('all').n_residues
atoms = u.select_atoms('all')
cl = u.select_atoms('name CL')
n = u.select_atoms('name N')
br = u.select_atoms('name BR')
na = u.select_atoms('name NA')
ow = u.select_atoms('name OW')
z_top = 399
z_bot = 395
resolution = 1000
A = 77.9
B = 77.9
C = 1500

def area(r):
    """ volume of a sphere of height z """
    area = sp.pi * r**2
    return area

def distance(a, b):
    """ get displacement in each coordinate and wrap w.r.t. lattice parameter """
    dx = abs(a[0] - b[0])
    x = min(dx, abs(A - dx))
    
    dy = abs(a[1] - b[1])
    y = min(dy, abs(B - dy))
    
    return np.sqrt(x**2 + y**2)

""" no reason to go above half of the smallest lattice parameter as mirror images start to be double-counted """
r_cutoff = A / 2
dr = r_cutoff / resolution
areas = np.zeros(resolution)
        
radii = np.linspace(0.0, resolution * dr, resolution)
g_of_r = np.zeros(resolution)


for ts in u.trajectory:
    """ isolate all ref molecules based on the position """
    
    anion_pos = n.positions
    data_anion = []
    for i in range(200):
        if z_bot < anion_pos[i][2] < z_top:
            data_anion.append(anion_pos[i])
    data_anion = np.array(data_anion)
    
    cation_pos = n.positions
    data_cation = []
    for i in range(200):
        if z_bot < cation_pos[i][2] < z_top:
            data_cation.append(cation_pos[i])
    data_cation = np.array(data_cation)
    
    
    for i, anion1 in enumerate(data_anion):
        for j in range(resolution):
            r1 = j * dr
            r2 = r1 + dr
            v1 = area(r1)
            v2 = area(r2)
            areas[j] += v2 - v1
            
        for anion2 in data_cation[i:]:
            dist = distance(anion1, anion2)
            index = int(dist / dr)
            if 0 < index < resolution:
                g_of_r[index] += 2.0
                
""" normalize by the volume of the spherical shell corresponding to each radius """
for i, value in enumerate(g_of_r):
    g_of_r[i] = value / areas[i]

plt.xlabel('r (Å)')
plt.ylabel('g$_{O-O}$(r)')
plt.plot(radii, g_of_r)
plt.show()

for i in range(1000):
    print(radii[i], g_of_r[i])